In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

In [2]:
def wrangle(filepath):
    df = pd.read_csv(filepath)
    
    # Split the 'category' column
    categories = df['category'].str.split('|', n=2, expand=True)
    categories.columns = ['category', 'secondaryCategory', 'tertiaryCategory']
    
    df = df.drop('category', axis=1)
    df = pd.concat([df, categories], axis=1)
    
    # Convert price columns to float
    df['discounted_price'] = df['discounted_price'].str.replace('₹', '').str.replace(',', '').astype(float)
    df['actual_price'] = df['actual_price'].str.replace('₹', '').str.replace(',', '').astype(float)

    # Clean and convert 'rating_count' to integer
    df['rating_count'] = df['rating_count'].replace('', np.nan).str.replace(',', '').astype(float)
    
    # Clean and convert 'rating' to float
    df['rating'] = df['rating'].replace('', np.nan).replace('|', '').replace('', np.nan)
    df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
    
    # Convert 'discount_percentage' to a decimal
    df['discount_percentage'] = df['discount_percentage'].str.replace('%', '').astype(float) / 100

    # Replace user_id with numeric only (unique)
    user_id_map = {old_id: new_id for new_id, old_id in enumerate(df['user_id'].unique())}
    df['user_id'] = df['user_id'].map(user_id_map)

    # Anonymize (& clean) user_name by keeping only the first part after the comma
    df['user_name'] = df['user_name'].apply(lambda x: x.split(',')[0] if ',' in x else x).apply(lambda x: x.split(' ')[0] if ' ' in x else x)

    # Split (& clean) review_id col
    df['review_id'] = df['review_id'].apply(lambda x: x.split(',')[0] if ',' in x else x)
    
    # Drop unnecessary columns
    df.drop(columns=['secondaryCategory', 'tertiaryCategory', 'img_link'], inplace=True)
    
    return df.dropna()

In [3]:
df = wrangle('data/amazon.csv')
df

,product_id,product_name,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,product_link,category
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,399.0,1099.0,0.64,4.2,24269.0,High Compatibility : Compatible With iPhone 12...,0,Manav,R3HXWT0LRP0NMF,"Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...,Computers&Accessories
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,199.0,349.0,0.43,4.0,43994.0,"Compatible with all Type C enabled devices, be...",1,ArdKn,RGIQEG07R9HS2,"A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://www.amazon.in/Ambrane-Unbreakable-Char...,Computers&Accessories
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,199.0,1899.0,0.90,3.9,7928.0,【 Fast Charger& Data Sync】-With built-in safet...,2,Kunal,R3J3EQQ9TZI5ZJ,"Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://www.amazon.in/Sounce-iPhone-Charging-C...,Computers&Accessories
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,329.0,699.0,0.53,4.2,94363.0,The boAt Deuce USB 300 2 in 1 cable is compati...,3,Omkar,R3EEUZKKK9J36I,"Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://www.amazon.in/Deuce-300-Resistant-Tang...,Computers&Accessories
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,154.0,399.0,0.61,4.2,16905.0,[CHARGE & SYNC FUNCTION]- This cable comes wit...,4,rahuls6099,R1BP4L2HH9TFUP,"As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://www.amazon.in/Portronics-Konnect-POR-1...,Computers&Accessories
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,B08L7J3T31,Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...,379.0,919.0,0.59,4.0,1090.0,SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...,1189,Prabha,R3G3XFHPBFF0E8,"Received the product without spanner,Excellent...","I received product without spanner,Excellent p...",https://www.amazon.in/Noir-Aqua-Spanner-Purifi...,Home&Kitchen
1461,B01M6453MB,Prestige Delight PRWO Electric Rice Cooker (1 ...,2280.0,3045.0,0.25,4.1,4118.0,"230 Volts, 400 watts, 1 Year",1190,Manu,R3DDL2UPKQ2CK9,"ok,everything was good couldn't return bcoz I ...","ok,got everything as mentioned but the measuri...",https://www.amazon.in/Prestige-Delight-PRWO-1-...,Home&Kitchen
1462,B009P2LIL4,Bajaj Majesty RX10 2000 Watts Heat Convector R...,2219.0,3080.0,0.28,3.6,468.0,International design and styling|Two heat sett...,1191,Nehal,R1TLRJVW4STY5I,"very good,Work but front melt after 2 month,Go...","plastic but cool body ,u have to find sturdy s...",https://www.amazon.in/Bajaj-RX-10-2000-Watt-Co...,Home&Kitchen
1463,B00J5DYCCA,Havells Ventil Air DSP 230mm Exhaust Fan (Pist...,1399.0,1890.0,0.26,4.0,8031.0,Fan sweep area: 230 MM ; Noise level: (40 - 45...,1192,Shubham,R39Q2Y79MM9SWK,"Fan Speed is slow,Good quality,Good product,go...",I have installed this in my kitchen working fi...,https://www.amazon.in/Havells-Ventilair-230mm-...,Home&Kitchen


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1462 entries, 0 to 1464
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product_id           1462 non-null   object 
 1   product_name         1462 non-null   object 
 2   discounted_price     1462 non-null   float64
 3   actual_price         1462 non-null   float64
 4   discount_percentage  1462 non-null   float64
 5   rating               1462 non-null   float64
 6   rating_count         1462 non-null   float64
 7   about_product        1462 non-null   object 
 8   user_id              1462 non-null   int64  
 9   user_name            1462 non-null   object 
 10  review_id            1462 non-null   object 
 11  review_title         1462 non-null   object 
 12  review_content       1462 non-null   object 
 13  product_link         1462 non-null   object 
 14  category             1462 non-null   object 
dtypes: float64(5), int64(1), object(9)
mem

In [5]:
user_dim = df[['user_id', 'user_name']]
user_dim.to_csv('data/DimUser.csv', index=False)
user_dim = pd.read_csv('data/DimUser.csv')
user_dim['user_email'] = user_dim['user_name'] + '@example.com'
user_dim.to_csv('data/DimUser.csv', index=False)

In [6]:
product_dim = df[['product_id', 'product_name', 'category', 'discounted_price', 'actual_price', 'about_product', 'rating', 'rating_count', 'product_link']].drop_duplicates()
product_dim.to_csv('data/DimProduct.csv', index=False)

In [7]:
review_dim = df[['review_id', 'review_title', 'review_content', 'product_id']]
review_dim.to_csv('data/DimReview.csv', index=False)

In [8]:
def generate_location_dim(n_records):
    # Define the states and cities
    states = ['Delhi', 'Maharashtra', 'Karnataka', 'Telangana', 'Tamil Nadu', 'Haryana', 'Uttar Pradesh']
    cities_by_state = {
        'Maharashtra': ['Mumbai', 'Thane', 'Pune'],
        'Karnataka': ['Bengaluru'],
        'Telangana': ['Hyderabad'],
        'Tamil Nadu': ['Chennai'],
        'Haryana': ['Gurgaon'],
        'Uttar Pradesh': ['Noida', 'Lucknow']
    }

    # Initialize the location data list
    location_dim = []

    # Generate synthetic LocationDim data
    for i in range(n_records):
        location_id = i + 1  # Unique numeric location_id
        state = random.choice(states)
        
        if state == 'Delhi':
            city = 'Delhi'
            pincode = random.randint(110000, 110099)
        else:
            city = random.choice(cities_by_state[state])
            match state:
                case 'Maharashtra':
                    match city:
                        case 'Mumbai':
                            pincode = random.randint(400001, 401107)
                        case 'Thane':
                            pincode = random.randint(400601, 401107)
                        case 'Pune':
                            pincode = random.randint(411001, 411048)
                case 'Karnataka':
                    if city == 'Bengaluru':
                        pincode = random.randint(560001, 560099)
                case 'Telangana':
                    if city == 'Hyderabad':
                        pincode = random.randint(500001, 500098)
                case 'Tamil Nadu':
                    if city == 'Chennai':
                        pincode = random.randint(600001, 600100)
                case 'Haryana':
                    if city == 'Gurgaon':
                        pincode = random.randint(122001, 122018)
                case 'Uttar Pradesh':
                    match city:
                        case 'Noida':
                            pincode = random.randint(201301, 201309)
                        case 'Lucknow':
                            pincode = random.randint(226001, 226016)
        
        # Append to location_dim list
        location_dim.append({
            'location_id': location_id,
            'city': city,
            'state': state,
            'pincode': pincode
        })

    return location_dim

In [9]:
def generate_dim_date(num_dates, start_date='2022-01-01', end_date='2022-02-01'):
    date_range = pd.date_range(start=start_date, end=end_date)
    random_dates = random.choices(date_range, k=num_dates)
    dim_date = []

    for i, date in enumerate(random_dates):
        date_id = i + 1  # Unique numeric date_id
        year = date.year
        quarter = (date.month - 1) // 3 + 1
        quartername = f'Q{quarter}'
        month = date.month
        monthname = date.strftime('%B')
        day = date.day
        weekday = date.weekday() + 1
        weekdayname = date.strftime('%A')
        
        dim_date.append({
            'date_id': date_id,
            'date': date,
            'year': year,
            'quarter': quarter,
            'quartername': quartername,
            'month': month,
            'monthname': monthname,
            'day': day,
            'weekday': weekday,
            'weekdayname': weekdayname
        })

    return pd.DataFrame(dim_date)

In [10]:
def generate_orders(num_orders, num_dates, num_locations, user_dim, product_dim):
    date_dim = generate_dim_date(num_dates)
    location_dim = generate_location_dim(num_locations)
    orders = []

    date_dim_list = date_dim['date_id'].tolist()
    location_dim_list = [loc['location_id'] for loc in location_dim]
    user_dim_list = user_dim['user_id'].tolist()
    product_dim_list = list(product_dim['product_id'])

    for i in range(num_orders):
        order_id = i + 1
        date_id = random.choice(date_dim_list)
        location_id = random.choice(location_dim_list)
        user_id = random.choice(user_dim_list)
        product_ids = random.sample(product_dim_list, random.randint(1, 5))
        amount = sum(product_dim[product_dim['product_id'].isin(product_ids)]['discounted_price'])
        returned = random.random() < 0.01

        orders.append({
            'order_id': order_id,
            'date_id': date_id,
            'location_id': location_id,
            'user_id': user_id,
            'product_ids': product_ids,
            'amount': amount,
            'returned': returned
        })

    # Convert to DataFrames
    orders_df = pd.DataFrame(orders)
    date_df = pd.DataFrame(date_dim)
    location_df = pd.DataFrame(location_dim)

    # Save to CSV files
    orders_df.to_csv('data/FactOrders.csv', index=False)
    date_df.to_csv('data/DimDate.csv', index=False)
    location_df.to_csv('data/DimLocation.csv', index=False)

    return orders_df, location_df, date_df


In [11]:
num_orders = 1462 * 4
num_dates = 31
num_locations = 1462

In [12]:
orders_df, location_df, date_df = generate_orders(num_orders, num_dates, num_locations, user_dim, product_dim)

In [13]:
orders_df

,order_id,date_id,location_id,user_id,product_ids,amount,returned
0,1,8,852,1013,[B07T5DKR5D],149.0,False
1,2,3,297,257,"[B08VRMK55F, B0B5KZ3C53, B0BK1K598K]",2676.0,False
2,3,13,1175,54,"[B0758F7KK7, B08H9Z3XQW, B097R45BH8]",6353.0,False
3,4,28,1049,0,"[B09MJ77786, B098R25TGC, B095JQVC7N, B08461VC1Z]",77296.0,False
4,5,9,1188,576,"[B0758F7KK7, B00LZLPYHW, B091KNVNS9, B082LSVT4B]",1402.0,False
...,...,...,...,...,...,...,...
5843,5844,7,498,1,"[B0993BB11X, B0798PJPCL]",2888.0,False
5844,5845,19,1322,659,"[B07N42JB4S, B09KLVMZ3B]",1276.0,False
5845,5846,31,779,218,"[B08RP2L2NL, B086PXQ2R4, B008P7IF02, B07RZZ1QSW]",5637.0,False
5846,5847,8,900,1026,"[B0746JGVDS, B07WDKLRM4]",14348.0,False


In [14]:
location_df

,location_id,city,state,pincode
0,1,Noida,Uttar Pradesh,201302
1,2,Gurgaon,Haryana,122004
2,3,Delhi,Delhi,110080
3,4,Pune,Maharashtra,411030
4,5,Chennai,Tamil Nadu,600068
...,...,...,...,...
1457,1458,Mumbai,Maharashtra,400142
1458,1459,Noida,Uttar Pradesh,201304
1459,1460,Hyderabad,Telangana,500051
1460,1461,Hyderabad,Telangana,500082


In [15]:
date_df

,date_id,date,year,quarter,quartername,month,monthname,day,weekday,weekdayname
0,1,2022-01-18,2022,1,Q1,1,January,18,2,Tuesday
1,2,2022-02-01,2022,1,Q1,2,February,1,2,Tuesday
2,3,2022-01-05,2022,1,Q1,1,January,5,3,Wednesday
3,4,2022-01-05,2022,1,Q1,1,January,5,3,Wednesday
4,5,2022-01-20,2022,1,Q1,1,January,20,4,Thursday
5,6,2022-01-02,2022,1,Q1,1,January,2,7,Sunday
6,7,2022-01-21,2022,1,Q1,1,January,21,5,Friday
7,8,2022-01-06,2022,1,Q1,1,January,6,4,Thursday
8,9,2022-01-02,2022,1,Q1,1,January,2,7,Sunday
9,10,2022-01-11,2022,1,Q1,1,January,11,2,Tuesday
